In [ ]:
import os

os.chdir(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))

os.environ["DEV_MOCK_CAMERA"] = "1"

In [ ]:
import logging
from datetime import datetime

import cv2
import pandas as pd
from matplotlib import pyplot as plt

from camera import get_camera

from processing import Frame

In [ ]:
logging.basicConfig(
    level=logging.WARNING,
    format="%(asctime)s %(levelname)s %(name)s: %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
)

In [ ]:
cam = get_camera()
cam.cam.set(cv2.CAP_PROP_POS_FRAMES, 0)
prev_frame = None
image = True
frames = []

while True:

    image = cam()

    if image is None:
        break

    frame = Frame(
        timestamp=datetime.now(),
        image=image,
        prev_frame=prev_frame,
    )
    frames.append(frame)
    prev_frame = frame

In [ ]:
df = pd.DataFrame(
    [
        {
            "hash": f.hash,
            "motion_prop": f.motion_mask.mean() / 255,
            "has_motion": bool(f.has_motion),
            "n_object_detections": len(f.object_detections),
        }
        for f in frames
    ]
)

In [ ]:
nframes = 15
ncols = 4

fig, ax_all = plt.subplots(nrows=nframes, ncols=ncols, figsize=(ncols * 4, nframes * 3))
ax_rows = iter(ax_all)

for counter, frame in enumerate(frames):

    if counter % 15 in [0]:

        ax_row = next(ax_rows, None)
        if ax_row is None:
            break
        ax_row[0].set_title(f"{counter}, {frame.hash}")
        ax_row[0].imshow(frame.image)

        ax_row[1].set_title(f"{frame.motion_mask.mean() / 255:.3f} {frame.has_motion}")
        ax_row[1].imshow(frame.motion_mask)

        if frame.has_motion:
            ax_row[2].set_title(frame.motion_bbox)
            ax_row[2].imshow(
                frame.image[
                    frame.motion_bbox[1] : frame.motion_bbox[3],
                    frame.motion_bbox[0] : frame.motion_bbox[2],
                ]
            )

        ax_row[3].set_title(
            f"{frame.has_excluded_class} {[x["class"] for x in frame.object_detections]}"
        )
        ax_row[3].imshow(frame.image_annotated)


fig.tight_layout()